In [1]:
from google.cloud import language_v1
import json, os

Below is the function that we want to deploy, but we need to try it locally to make sure it works

In [4]:
def get_sentiment(request):

    try:
        
        project_id = os.environ.get("GOOG_PROJECT")
        
        # Get the text from the request body
        request_json = request.get_json(silent=True)
        if request_json and 'text' in request_json:
            text = request_json['text']
        else:
            return ("Please provide 'text' in the request body.", 400)


        # Instantiates a client.
        client = language_v1.LanguageServiceClient()

        # The text to analyze.
        #text = "not sure that this movie was all that good."
        document = language_v1.types.Document(
            content=text, type_=language_v1.types.Document.Type.PLAIN_TEXT
        )

        # Detects the sentiment of the text.
        sentiment = client.analyze_sentiment(
            request={"document": document}
        ).document_sentiment

        #print(f"Text: {text}")
        #print(f"Sentiment: {sentiment.score}, {sentiment.magnitude}")
        if sentiment.score < 0:
            outcome = "Negative"
        elif sentiment.score > 0:
            outcome = "Positive"
        else:
            outcome = "Neutral"


        # Serialize the results to JSON
        return json.dumps(outcome, indent=2), 200, {'Content-Type': 'application/json'}  

    except Exception as e:
        # Handle errors 
        print(f"Error: {e}")
        return f"An error occurred: {e}", 500

Following will test the fuction

In [7]:
if __name__ == '__main__':
    # Simulate a request object
    class MockRequest:
        def get_json(self, silent=True):
            return {
                'text': "i love ice cream!"  
            }

    request = MockRequest()
    response = get_sentiment(request)  
    print(f"Response: {response}")
    print(json.dumps(response, indent=2))

Response: ('"Positive"', 200, {'Content-Type': 'application/json'})
[
  "\"Positive\"",
  200,
  {
    "Content-Type": "application/json"
  }
]
